Se importan todas las librerías necesarias.

In [21]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image,ImageDraw, ImageFont
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,9)
plt.style.use('ggplot')

Se importa el dataset 

In [4]:
path = 'https://www.juanbarrios.com/archivos/cardiopatia.csv'
dataframe = pd.read_csv(path)
dataframe.head()

,cardiopatia,Sexo,Tabaco,HTA,Diabetes,obesidad,edad_grup
0,2,2,3,2,2,1,4
1,1,1,2,2,2,2,4
2,2,2,3,1,1,2,4
3,2,2,2,1,1,2,4
4,1,2,1,2,2,2,3


Conocemos el tipo de atributos del dataset. En este caso, hay 3 atributos tipo int(numéricos) y 4 de tipo objeto.

In [6]:
dataframe.dtypes

cardiopatia     int64
Sexo            int64
Tabaco         object
HTA            object
Diabetes       object
obesidad       object
edad_grup       int64
dtype: object

Los campos de tipo Object son debidos a que a la hora de incluir los datos hay espacios, los cuales se deben de convertir a valores 0 y luego convertir a tipo numerico para poder operar con ellos.

In [11]:
dataframe=dataframe.replace(r'\s+',0,regex=True)
dataframe['Tabaco']=dataframe['Tabaco'].astype(int)
dataframe['HTA']=dataframe['HTA'].astype(int)
dataframe['Diabetes']=dataframe['Diabetes'].astype(int)
dataframe['obesidad']=dataframe['obesidad'].astype(int)
dataframe.describe()

,cardiopatia,Sexo,Tabaco,HTA,Diabetes,obesidad,edad_grup
count,80.000000,80.00000,80.000000,80.000000,80.000000,80.000000,80.00000
mean,1.612500,1.67500,1.875000,1.437500,1.812500,1.525000,3.40000
std,0.490253,0.47133,0.919204,0.547578,0.452664,0.594809,0.62844
min,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,2.00000
25%,1.000000,1.00000,1.000000,1.000000,2.000000,1.000000,3.00000
50%,2.000000,2.00000,2.000000,1.000000,2.000000,2.000000,3.00000
75%,2.000000,2.00000,3.000000,2.000000,2.000000,2.000000,4.00000
max,2.000000,2.00000,3.000000,2.000000,2.000000,2.000000,4.00000


# Regresión logística

Se le indica al algoritmo que nuestra variable predictiva es 'cardiopatia'. El sistema nos muestra 80 casos con 6 atributos sin contar la variable predictiva.

In [12]:
X = np.array(dataframe.drop(['cardiopatia'],1))
y = np.array(dataframe['cardiopatia'])
X.shape

(80, 6)

Se aplica el modelo. En este caso, regresión logística (algoritmo de clasificación) con un peso entre las clases de  1:1.633

In [14]:
model = linear_model.LogisticRegression(class_weight={1:1.633})
model.fit(X,y)

LogisticRegression(class_weight={1: 1.633})

Se obtiene el Índice de predicción de nuestro modelo.

In [15]:
model.score(X,y)

0.675

Se observa que el rendimiento es malo, por tanto, se prueba con otro algoritmo

# Arbol de decisión

Se excluye la variable predictiva

In [18]:
entradas = dataframe.drop(['cardiopatia'], axis= 1)
entradas.columns

Index(['Sexo', 'Tabaco', 'HTA', 'Diabetes', 'obesidad', 'edad_grup'], dtype='object')

3 parámetros que es necesario “ajustar”:

    min_samples _split:  Es el numero de muestras antes de dividirse el árbol en una nueva rama

    min_samples _leaf:  Es el número de muestras minino que debe tener cada hoja al final = 5 par este caso tratandose de que son muy pocos casos

    max_depth:  Es el número de niveles que tendrá nuestro  el árbol = 2 par este caso, tratándose de que son muy pocos casos

    class_weight  se refiere al desbalance que hubiese que corregir por desbalance entre las clases en nuestro caso 1:1.6333 ( como se había observado antes)

In [27]:
y_train = dataframe['cardiopatia']
x_train = dataframe.drop(['cardiopatia'],axis=1).values

#Parametros del arbol de decision
decision_tree = tree.DecisionTreeClassifier(criterion='entropy',min_samples_split=5,min_samples_leaf=2,max_depth = 7, class_weight={1:1.6333})

decision_tree.fit(x_train, y_train)


DecisionTreeClassifier(class_weight={1: 1.6333}, criterion='entropy',
                       max_depth=7, min_samples_leaf=2, min_samples_split=5)

Se exporta el modelo a formato .dot y este se convierte en png para poder visualizarlo

In [44]:
shell=True
with open(r'tree.dot','w') as f:
    f = tree.export_graphviz(decision_tree,
                            out_file=f,
                            max_depth= 7,
                            impurity = True,
                            feature_names = list(dataframe.drop(['cardiopatia'],axis=1)),
                            class_names= ['1: Sano/ Isquemia','2: Infarto Agudo'],
                            rounded = True,
                            filled= True)
check_call(['/home/ptolsa/Escritorio/Busqueda_datasets_definicion_casos_uso/','-Tpng',r'tree.dot','-o',r'tree.png'])
PImage("tree.png")

PermissionError: [Errno 13] Permission denied: '/home/ptolsa/Escritorio/Busqueda_datasets_definicion_casos_uso/'

In [37]:
pwd

'/home/ptolsa/Escritorio/Busqueda_datasets_definicion_casos_uso'

Se calcula la precisión del modelo.

In [28]:
decision_tree.score(x_train,y_train)

0.825

In [29]:
y_pred= decision_tree.predict(x_train)

In [30]:
print(confusion_matrix(y,y_pred))

[[27  4]
 [10 39]]


La matriz de confusión muestra 14 casos en donde el modelo falló 14 de 80 = 0.175 esto equivale a un error del 17%.

In [31]:
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           1       0.73      0.87      0.79        31
           2       0.91      0.80      0.85        49

    accuracy                           0.82        80
   macro avg       0.82      0.83      0.82        80
weighted avg       0.84      0.82      0.83        80

